In [ ]:
!pip install transformers torch geopy folium sentencepiece

In [ ]:
from transformers import pipeline
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import folium
from time import sleep

In [ ]:
geolocator = Nominatim(user_agent="crisislens_hack")

In [ ]:
classifier = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    return_all_scores=True
)

In [ ]:
reports = [
    {"text": "House collapsed on Gandhi Street! My brother is trapped!! #ChennaiFlood", "source": "Twitter"},
    {"text": "No electricity or water in Adyar. Elderly couple needs insulin.", "source": "Telegram"},
    {"text": "Relief camp at Teynampet School — running low on food and blankets.", "source": "News"},
    {"text": "Road to Kottivakkam blocked by fallen trees. Ambulance can't pass.", "source": "Twitter"}
]

In [ ]:
def get_urgency_score(text):
    scores = classifier(text)[0]
    try:
        neg_score = next(s['score'] for s in scores if s['label'] == 'negative')
    except StopIteration:
        neg_score = min(s['score'] for s in scores)
    return neg_score

def geocode_location(text):
    locations = [
        "Chennai", "Gandhi Street, Chennai", "Adyar, Chennai",
        "Teynampet, Chennai", "Kottivakkam, Chennai"
    ]
    for loc in locations:
        if loc.split(',')[0].lower() in text.lower():
            try:
                return geolocator.geocode(loc, timeout=10)
            except GeocoderTimedOut:
                continue
    return geolocator.geocode("Chennai, India")

In [ ]:
processed = []
for r in reports:
    urgency = get_urgency_score(r["text"])
    loc = geocode_location(r["text"])

    life_threat_keywords = ["trapped", "buried", "collapsed", "bleeding", "unconscious", "crushed"]
    if any(kw in r["text"].lower() for kw in life_threat_keywords):
        urgency = max(urgency, 0.95)

    if loc:
        if urgency > 0.9:
            color = "red"
        elif urgency > 0.6:
            color = "orange"
        else:
            color = "green"

        processed.append({
            "text": r["text"],
            "urgency": urgency,
            "lat": loc.latitude,
            "lon": loc.longitude,
            "color": color
        })
    sleep(1)


In [ ]:
processed

In [ ]:
m = folium.Map(location=[13.0827, 80.2707], zoom_start=12)

# Add pins
for item in processed:
    folium.CircleMarker(
        location=[item["lat"], item["lon"]],
        radius=10,
        popup=f"<b>{item['color'].upper()} • Urgency: {item['urgency']:.2f}</b><br><i>→ {'Keyword boost' if item['urgency'] >= 0.95 else 'Model score'}</i><br>{item['text']}",
        color=item["color"],
        fill=True,
        fill_color=item["color"]
    ).add_to(m)

In [ ]:
m